In [ ]:
%%script false --no-raise-error
import json
from google.colab import userdata
from google.oauth2 import service_account
from google.cloud.bigquery import magics

credentials_json = userdata.get('BIGQUERY_CREDENTIALS')
credentials = service_account.Credentials.from_service_account_info(json.loads(credentials_json))
magics.context.credentials = credentials

Couldn't find program: 'false'


In [ ]:
from google.cloud import bigquery
from google.cloud.bigquery import magics
%load_ext bigquery_magics

data_set = "testing_set2"
project_name = "emerald-entity-468916-f9"

job_config = bigquery.QueryJobConfig(default_dataset = f"{project_name}.{data_set}")
client = bigquery.Client(project = project_name, default_query_job_config = job_config, credentials = globals().get('credentials', None))
magics.context.default_query_job_config = job_config
magics.context.project = project_name

In [3]:
%%bigquery --pyformat
CREATE OR REPLACE PROCEDURE phase2_correction_correct()
BEGIN
    DECLARE correct_model_params JSON DEFAULT JSON '''
    {{"systemInstruction": {{"parts": [{{"text": "You are an assistant helping to fix the content of scanned books."}}]}}}}
    ''';

    UPDATE {data_set}.tmp_correction_chunks b SET b.corrected_txt = 
        AI.GENERATE(FORMAT(p.prompt, b.prefix, b.original_txt, b.suffix), 
        connection_id => 'us.ai_connection', endpoint => 'gemini-2.5-flash-lite',
        model_params => correct_model_params).result
    FROM {data_set}.prompts p WHERE b.corrected_txt IS NULL AND p.code = 'correct';

    MERGE {data_set}.books b
    USING (SELECT book_id, STRING_AGG(corrected_txt, ' ' ORDER BY chunk_number) AS aggregated_txt
    FROM {data_set}.tmp_correction_chunks GROUP BY book_id) cb
    ON b.book_id = cb.book_id AND b.corrected_txt IS NULL
    WHEN MATCHED THEN UPDATE SET corrected_txt = cb.aggregated_txt;
END;

Query is running:   0%|          |

""


In [21]:
%%bigquery --pyformat
CREATE OR REPLACE PROCEDURE phase3_prepare_chunks()
BEGIN
    create or replace table {data_set}.tmp_overlapped_chunks AS
    SELECT book_id, chunk_num - 1 as chunk_number, SUBSTR(corrected_txt, (chunk_num - 1) * 40000 + 1, 40000) as txt, CAST(null AS STRING) text_with_overlap
    FROM {data_set}.books, UNNEST(GENERATE_ARRAY(1, CAST(CEIL(LENGTH(corrected_txt) / 40000) AS INT64))) as chunk_num
    WHERE processed = False or processed is null;

    UPDATE {data_set}.tmp_overlapped_chunks curr set curr.text_with_overlap = curr.txt || ' ' || RIGHT(prev.txt, 4000)
    FROM {data_set}.tmp_overlapped_chunks prev WHERE curr.book_id = prev.book_id AND curr.chunk_number = prev.chunk_number + 1;

    UPDATE {data_set}.tmp_overlapped_chunks set text_with_overlap = txt where text_with_overlap is null;

    CREATE OR REPLACE TABLE {data_set}.chunks(book_id STRING, chunk_number INTEGER, txt STRING, summary STRING,
        fragment_number STRING, characters_id_data STRING, characters_full_data STRING);

    INSERT INTO {data_set}.chunks (book_id, chunk_number, txt)
    (SELECT book_id, chunk_number, text_with_overlap FROM {data_set}.tmp_overlapped_chunks
    EXCEPT DISTINCT SELECT book_id, chunk_number, txt FROM {data_set}.chunks);
END;

Query is running:   0%|          |

""


In [5]:
%%bigquery --pyformat

CREATE OR REPLACE PROCEDURE phase4_fragments_summarization()
BEGIN
  	UPDATE {data_set}.chunks c SET summary = 
		coalesce(AI.GENERATE(FORMAT(p.prompt, s.size, c.txt), 
		connection_id => 'us.ai_connection', endpoint => 'gemini-2.5-flash-lite').result, '')
    FROM (select book_id, cast(80000 / count(*) as INT64) size from {data_set}.chunks group by book_id) s, {data_set}.prompts p
    WHERE c.book_id = s.book_id AND c.summary IS NULL AND p.code = 'summarize';
END;

Query is running:   0%|          |

""


In [6]:
%%bigquery --pyformat
CREATE OR REPLACE PROCEDURE phase4_summarize_fragments_summaries()
BEGIN
	UPDATE {data_set}.books b SET b.summary = coalesce(AI.GENERATE(FORMAT(p.prompt, s.summary), 
			connection_id => 'us.ai_connection', endpoint => 'gemini-2.5-flash-lite').result, '')
	FROM {data_set}.prompts p,
        (SELECT book_id, STRING_AGG(summary, '/n' ORDER BY chunk_number) summary FROM {data_set}.chunks GROUP BY book_id) s 
    WHERE p.code = 'reduce_summary' AND b.book_id = s.book_id AND b.summary is null;
END;

Query is running:   0%|          |

""


In [ ]:
%%bigquery --pyformat
CREATE OR REPLACE PROCEDURE phase5_prepare_character_ids_from_fragments()
BEGIN
    UPDATE {data_set}.chunks SET fragment_number = FORMAT('%03d', chunk_number + 1) where fragment_number is null;

    UPDATE {data_set}.chunks c SET c.characters_id_data = 
        AI.GENERATE(FORMAT(p.prompt, c.txt), connection_id => 'us.ai_connection', endpoint => 'gemini-2.5-flash-lite').result
    FROM {data_set}.prompts p, {data_set}.books b
    WHERE c.characters_full_data is null AND p.code = 'characters_id_data' and c.book_id = b.book_id;

    CREATE OR REPLACE TABLE {data_set}.tmp_characters_id_data AS
    (select book_id, chunk_number, fragment_number, x as characters_id_data,
    CAST(NULL AS STRING) full_name, CAST(NULL AS STRING) information, CAST(NULL AS INT64) importance
    from {data_set}.chunks, UNNEST(JSON_QUERY_ARRAY(TRIM(characters_id_data, '`json\n'), '$')) x);

    CREATE OR REPLACE TABLE {data_set}.identifiers AS
    SELECT row_number() OVER(ORDER BY book_id, chunk_number, full_name) id, * from {data_set}.tmp_characters_id_data;

    UPDATE {data_set}.identifiers SET full_name = json_value(characters_id_data, '$.full_name'), information = json_value(characters_id_data, '$.information'),
        importance = CAST(JSON_VALUE(characters_id_data, '$.importance') AS INT64) where full_name is null;

    UPDATE {data_set}.identifiers SET importance = 1 where importance = 0 or importance is null;
END;


Query is running:   0%|          |

""


In [8]:
%%bigquery --pyformat
CREATE OR REPLACE PROCEDURE phase5_prepare_character_ids_initial_embeddings()
BEGIN
  create or replace table {data_set}.tmp_characters_id_embeddings as SELECT * FROM ML.GENERATE_EMBEDDING(
    MODEL `{data_set}.gemini-embedding-001`,
    (SELECT characters_id_data as content, full_name as title, id, book_id, chunk_number, FALSE is_query FROM {data_set}.identifiers),
    STRUCT(TRUE AS flatten_json_output, 'RETRIEVAL_DOCUMENT' as task_type));
END;

Query is running:   0%|          |

""


In [9]:
%%bigquery --pyformat
CREATE OR REPLACE PROCEDURE phase5_merge_characters_duplicates_with_return_param(OUT updated INT64)  
BEGIN
	create or replace table {data_set}.tmp_characters_id_distance as
  SELECT query.id query_id, query.title query_full_name, base.id, base.title full_name, base.chunk_number, base.book_id, distance FROM
  VECTOR_SEARCH(
	(SELECT id, book_id, chunk_number, title, ml_generate_embedding_result FROM {data_set}.tmp_characters_id_embeddings),
	'ml_generate_embedding_result',
	(SELECT id, title, ml_generate_embedding_result FROM {data_set}.tmp_characters_id_embeddings),
	'ml_generate_embedding_result',
	top_k => 2);

  delete from {data_set}.tmp_characters_id_distance where query_id = id;
  delete from {data_set}.tmp_characters_id_distance d where d.id < d.query_id and exists (select 1 from {data_set}.tmp_characters_id_distance dd where dd.id = d.query_id and d.id = dd.query_id);

  create or replace table {data_set}.tmp_characters_duplicate_candidates as
  select string_agg(pair, ',\n') pairs, book_id, chunk_number from
  (select TO_JSON_STRING(STRUCT(query.id, query.full_name as first_individual_full_name, query.information as first_individual_information, 
  base.full_name as second_individual_full_name, base.information as second_individual_information), true) pair, embed.book_id, query.chunk_number, query.fragment_number
  from {data_set}.tmp_characters_id_distance embed
  JOIN {data_set}.identifiers query ON embed.query_id = query.id
  JOIN {data_set}.identifiers base ON embed.id = base.id)
  group by book_id, chunk_number;

  create or replace table {data_set}.tmp_the_same_characters as SELECT pairs.book_id, pairs.chunk_number,
  AI.GENERATE(FORMAT(p.prompt, pairs.pairs, cb.summary), connection_id => 'us.ai_connection', endpoint => 'gemini-2.5-flash-lite').result pairs
  FROM {data_set}.prompts p, {data_set}.tmp_characters_duplicate_candidates pairs, {data_set}.books cb
  WHERE p.code = 'find_the_same_characters' and pairs.book_id = cb.book_id;

  create or replace table {data_set}.tmp_different_characters as SELECT pairs.book_id, pairs.chunk_number,
  AI.GENERATE(FORMAT(p.prompt, pairs.pairs, cb.summary), connection_id => 'us.ai_connection', endpoint => 'gemini-2.5-flash-lite').result pairs
  FROM {data_set}.prompts p, {data_set}.tmp_the_same_characters pairs, {data_set}.books cb
  WHERE p.code = 'find_different_characters' and pairs.book_id = cb.book_id;

  CREATE OR REPLACE TABLE {data_set}.tmp_the_same_characters_split AS
  select book_id, chunk_number, pair, CAST(NULL AS INT64) id
  from {data_set}.tmp_the_same_characters, UNNEST(JSON_QUERY_ARRAY(TRIM(pairs, '`json\n'), '$')) pair;		
  UPDATE {data_set}.tmp_the_same_characters_split SET id = CAST(JSON_VALUE(pair, '$.id') AS INT64) where id is null;

  CREATE OR REPLACE TABLE {data_set}.tmp_different_characters_split AS
  select book_id, chunk_number, pair, CAST(NULL AS INT64) id
  from {data_set}.tmp_different_characters, UNNEST(JSON_QUERY_ARRAY(TRIM(pairs, '`json\n'), '$')) pair;		
  UPDATE {data_set}.tmp_different_characters_split SET id = CAST(JSON_VALUE(pair, '$.id') AS INT64) where id is null;

  DELETE FROM {data_set}.tmp_the_same_characters_split WHERE ID IN (SELECT id FROM {data_set}.tmp_different_characters_split);		
  DELETE FROM {data_set}.tmp_characters_id_distance where query_id NOT IN (SELECT id from {data_set}.tmp_the_same_characters_split);

  create or replace table {data_set}.tmp_bidirectional_edges as (
  SELECT query_id, id FROM {data_set}.tmp_characters_id_distance UNION DISTINCT SELECT id, query_id FROM {data_set}.tmp_characters_id_distance);

  create or replace table {data_set}.tmp_grouped_duplicates_graph as
  WITH RECURSIVE connected_components AS (
      SELECT query_id AS node, query_id AS root, 0 AS iteration FROM {data_set}.tmp_bidirectional_edges
      UNION ALL
      SELECT edges.id AS node, LEAST(comp.root, edges.id) AS root, comp.iteration + 1
      FROM connected_components comp INNER JOIN {data_set}.tmp_bidirectional_edges edges ON comp.node = edges.query_id
      WHERE comp.iteration < 10
  ),
  unique_graph AS (SELECT node, MIN(root) AS group_id FROM connected_components GROUP BY node)
  SELECT group_id, ARRAY_AGG(DISTINCT node ORDER BY node) AS duplicate_ids_array FROM unique_graph GROUP BY group_id;

  create or replace table {data_set}.tmp_grouped_duplicates_graph_enriched as
  WITH duplicates AS (select group_id, id from {data_set}.tmp_grouped_duplicates_graph, UNNEST(duplicate_ids_array) id)
  SELECT d.group_id, "[\n" || STRING_AGG(TO_JSON_STRING(STRUCT(i.full_name, i.information), true), ",\n" ORDER BY i.chunk_number) || "\n]" duplicated_ids
  FROM {data_set}.identifiers i join duplicates d on i.id = d.id group by group_id;

  create or replace table {data_set}.tmp_merged_duplicates_double_check as SELECT x.group_id,
  AI.GENERATE_BOOL(FORMAT(p.prompt, x.duplicated_ids, b.summary), connection_id => 'us.ai_connection', endpoint => 'gemini-2.5-flash-lite').result check
  FROM {data_set}.prompts p, {data_set}.tmp_grouped_duplicates_graph_enriched x, {data_set}.identifiers i, {data_set}.books b WHERE p.code = 'merge_character_ids_double_check' and i.id = x.group_id and i.book_id = b.book_id;

  delete from {data_set}.tmp_grouped_duplicates_graph WHERE group_id IN (
    select group_id from {data_set}.tmp_merged_duplicates_double_check where check = FALSE);

  delete from {data_set}.tmp_grouped_duplicates_graph_enriched WHERE group_id IN (
    select group_id from {data_set}.tmp_merged_duplicates_double_check where check = FALSE);

  create or replace table {data_set}.tmp_merged_duplicates as SELECT x.group_id,
  AI.GENERATE(FORMAT(p.prompt, x.duplicated_ids, b.summary), connection_id => 'us.ai_connection', endpoint => 'gemini-2.5-flash-lite').result character_id
  FROM {data_set}.prompts p, {data_set}.tmp_grouped_duplicates_graph_enriched x, {data_set}.identifiers i, {data_set}.books b WHERE p.code = 'merge_character_ids' and i.id = x.group_id and i.book_id = b.book_id;

  CREATE OR REPLACE TABLE {data_set}.tmp_merged_duplicates_split AS
  select group_id, TRIM(character_id, '`json\n') character_id, CAST(NULL AS STRING) full_name, CAST(NULL AS STRING) information
  from {data_set}.tmp_merged_duplicates;
  UPDATE {data_set}.tmp_merged_duplicates_split SET full_name = json_value(character_id, '$.full_name'), information = json_value(character_id, '$.information') where full_name is null;

  UPDATE {data_set}.identifiers chid SET chid.full_name = duplicate.full_name, chid.information = duplicate.information,
  chid.characters_id_data = TO_JSON_STRING(STRUCT(duplicate.full_name, duplicate.information), true)
  FROM {data_set}.tmp_merged_duplicates_split duplicate WHERE chid.id = duplicate.group_id;

  create or replace table {data_set}.tmp_grouped_ids_split as select group_id, id from {data_set}.tmp_grouped_duplicates_graph, UNNEST(duplicate_ids_array) id;

  UPDATE {data_set}.identifiers chid SET chid.importance = dp.importance, chid.fragment_number = dp.fragment_number
  FROM (select chid.id, sum(di.importance) importance, string_agg(distinct di.fragment_number, ',') fragment_number
  FROM {data_set}.identifiers chid JOIN {data_set}.tmp_grouped_ids_split d ON d.group_id = chid.id
  JOIN {data_set}.identifiers di ON d.id = di.id group by chid.id) dp
  WHERE chid.id = dp.id;

  delete from {data_set}.tmp_grouped_ids_split where group_id = id;
  DELETE FROM {data_set}.identifiers WHERE id IN (SELECT id from {data_set}.tmp_grouped_ids_split);
  DELETE FROM {data_set}.tmp_characters_id_embeddings where id IN (SELECT id from {data_set}.tmp_grouped_ids_split);

  create or replace table {data_set}.tmp_characters_id_changed_embeddings as SELECT * FROM ML.GENERATE_EMBEDDING(
    MODEL `{data_set}.gemini-embedding-001`,
    (SELECT chid.characters_id_data as content, chid.full_name as title, chid.id FROM {data_set}.identifiers chid
    JOIN {data_set}.tmp_grouped_duplicates_graph duplicate ON chid.id = duplicate.group_id),
    STRUCT(TRUE AS flatten_json_output, 'RETRIEVAL_DOCUMENT' as task_type));

  UPDATE {data_set}.tmp_characters_id_embeddings t SET ml_generate_embedding_result = tnew.ml_generate_embedding_result, ml_generate_embedding_statistics = tnew.ml_generate_embedding_statistics,
    ml_generate_embedding_status = tnew.ml_generate_embedding_status, title = tnew.title, content = tnew.content
  FROM {data_set}.tmp_characters_id_changed_embeddings tnew WHERE tnew.id = t.id;

  SET updated = @@row_count;
END;

Query is running:   0%|          |

""


In [10]:
%%bigquery --pyformat
CREATE OR REPLACE PROCEDURE phase5_merge_characters_duplicates()  
BEGIN
  DECLARE merged INT64 DEFAULT 0;
  call {data_set}.phase5_merge_characters_duplicates_with_return_param(merged);
END;

Query is running:   0%|          |

""


In [11]:
%%bigquery --pyformat
CREATE OR REPLACE PROCEDURE phase5_rebuild_indentifier_jsons()  
BEGIN
    UPDATE {data_set}.identifiers SET characters_id_data = TO_JSON_STRING(STRUCT(id, full_name, information), true) WHERE full_name is not null;
END;

Query is running:   0%|          |

""


In [12]:
%%bigquery --pyformat
CREATE OR REPLACE PROCEDURE phase6_gather_characters_full_data_from_chunks()  
BEGIN
    UPDATE {data_set}.chunks ecb SET ecb.characters_full_data = 
    coalesce(AI.GENERATE(FORMAT(
            p.prompt, 
            (SELECT "[\n" || STRING_AGG(characters_id_data, ",\n") || "\n]" FROM {data_set}.identifiers WHERE fragment_number LIKE '%' || ecb.fragment_number || '%' AND book_id = b.book_id GROUP BY book_id), 
            ecb.txt, b.summary), 
        connection_id => 'us.ai_connection', endpoint => 'gemini-2.5-flash-lite',
        model_params => JSON '{{"systemInstruction": {{"parts": [{{"text": "You are an expert in extracting information from books."}}]}}}}').result, '[]')
    FROM {data_set}.prompts p, {data_set}.books b
    WHERE ecb.characters_full_data is null AND p.code = 'extract_data' and ecb.book_id = b.book_id;

    UPDATE {data_set}.chunks b SET b.characters_full_data = 
    TRIM(AI.GENERATE(FORMAT(p.prompt, 'array', b.characters_full_data), connection_id => 'us.ai_connection', endpoint => 'gemini-2.5-flash-lite').result, '`json\n')
    FROM {data_set}.prompts p WHERE b.characters_full_data is not null AND p.code = 'json';
END;

Query is running:   0%|          |

""


In [13]:
%%bigquery --pyformat
CREATE OR REPLACE PROCEDURE phase6_group_the_same_characters_data_for_merging()  
BEGIN
    CREATE OR REPLACE TABLE {data_set}.tmp_split_full_data as
    select book_id, character_full_data, chunk_number, CAST(NULL AS INT64) id, CAST(NULL AS STRING) full_name from {data_set}.chunks,
    UNNEST(JSON_QUERY_ARRAY(characters_full_data, '$')) character_full_data where characters_full_data is not null;

    UPDATE {data_set}.tmp_split_full_data SET full_name = json_value(character_full_data, '$.full_name'), id = CAST(JSON_VALUE(character_full_data, '$.id') AS INT64) WHERE full_name IS NULL;

    CREATE OR REPLACE TABLE {data_set}.tmp_grouped_full_data AS
    SELECT id, book_id, count(id) as parts, "[\n" || STRING_AGG(character_full_data, ",\n" ORDER BY chunk_number) || "\n]" full_data_array,
    CAST(NULL AS STRING) character_full_data,  CAST(NULL AS STRING) information
    FROM {data_set}.tmp_split_full_data GROUP BY book_id, id;
END;

Query is running:   0%|          |

""


In [14]:
%%bigquery --pyformat
CREATE OR REPLACE PROCEDURE phase6_merge_data()
BEGIN
    UPDATE {data_set}.tmp_grouped_full_data data SET data.character_full_data = 
    AI.GENERATE(FORMAT(p.prompt, data.full_data_array, cb.SUMMARY, ids.information), connection_id => 'us.ai_connection', endpoint => 'gemini-2.5-flash-lite').result
    FROM {data_set}.prompts p, {data_set}.books cb, {data_set}.identifiers ids
    WHERE data.character_full_data is null AND data.full_data_array is not null and data.parts > 1
    AND p.code = 'merge_character' and data.book_id = cb.book_id and data.book_id = ids.book_id AND ids.id = data.id;

    UPDATE {data_set}.tmp_grouped_full_data SET character_full_data = TRIM(full_data_array, '[] \n') 
    where character_full_data is null AND full_data_array is not null and parts = 1;

    UPDATE {data_set}.tmp_grouped_full_data data SET data.character_full_data = 
    TRIM(AI.GENERATE(FORMAT(p.prompt, data.character_full_data), connection_id => 'us.ai_connection', endpoint => 'gemini-2.5-flash-lite').result, '`json\n')
    FROM {data_set}.prompts p WHERE data.character_full_data is not null AND p.code = 'json_final_check';

    UPDATE {data_set}.tmp_grouped_full_data a SET a.information = ids.information
    FROM {data_set}.identifiers ids WHERE ids.id = a.id;
END;

Query is running:   0%|          |

""


In [15]:
%%bigquery --pyformat
CREATE OR REPLACE PROCEDURE phase6_copy_final_data()
BEGIN
  INSERT INTO {data_set}.characters (book_id, id, full_name, sex, social_class, mental_health, intelligence, wealth, values, information)
  select
    book_id,
    id,
    json_value(character_full_data, '$.full_name') full_name,
    json_value(character_full_data, '$.sex') sex,
    json_value(character_full_data, '$.social_class') social_class,
    json_value(character_full_data, '$.mental_health') mental_health,
    json_value(character_full_data, '$.intelligence') intelligence,
    json_value(character_full_data, '$.wealth') wealth,
    json_value(character_full_data, '$.values') values,
    information
  from {data_set}.tmp_grouped_full_data a where not exists (
    select 1 from {data_set}.characters fa where a.id = fa.id AND a.book_id = fa.book_id
  );

  UPDATE {data_set}.characters fa SET fa.importance = ids.importance
  FROM {data_set}.identifiers ids where ids.id = fa.id AND ids.book_id = fa.book_id AND fa.importance is null;

  update {data_set}.books set processed = TRUE where book_id IN (
    SELECT book_id from {data_set}.tmp_grouped_full_data);
END;

Query is running:   0%|          |

""


In [16]:
%%bigquery --pyformat
CREATE OR REPLACE PROCEDURE phase7_split_traits(trait_desc STRING)
BEGIN
    create or replace table {data_set}.tmp_split_traits as 
    SELECT t.book_id, t.id, t.traits as original_traits,
    coalesce(AI.GENERATE(FORMAT(p.prompt, trait_desc, t.traits, i.information), connection_id => 'us.ai_connection', endpoint => 'gemini-2.5-flash-lite').result, '[]') traits
    FROM {data_set}.prompts p, {data_set}.tmp_copied_traits t, {data_set}.characters i
    WHERE p.code = 'split_traits' AND t.id = i.id AND t.book_id = i.book_id;

    CREATE OR REPLACE TABLE {data_set}.tmp_split_traits_final AS
    SELECT *, row_number() OVER(ORDER BY id, book_id) trait_id, CAST(NULL AS INT64) cluster_id FROM
    (select book_id, id, trait
    from {data_set}.tmp_split_traits, UNNEST(JSON_QUERY_ARRAY(TRIM(traits, '`json\n'), '$')) as trait);	

    update {data_set}.tmp_split_traits_final set trait = TRIM(trait, '"') where 1=1;
END;

Query is running:   0%|          |

""


In [17]:
%%bigquery --pyformat
CREATE OR REPLACE PROCEDURE phase7_identify_clusters(clusters INT64)
BEGIN
    create or replace table {data_set}.tmp_traits_embeddings as SELECT * FROM ML.GENERATE_EMBEDDING(
    MODEL `{data_set}.gemini-embedding-001`,
    (SELECT trait as content, trait_id FROM {data_set}.tmp_split_traits_final),
    STRUCT(TRUE AS flatten_json_output, 'SEMANTIC_SIMILARITY' as task_type, 128 as OUTPUT_DIMENSIONALITY));

    create or replace table {data_set}.tmp_traits_embeddings_modeling_sample as
    WITH size as (select count(*) count from {data_set}.tmp_traits_embeddings)
    SELECT ml_generate_embedding_result, content, trait_id FROM {data_set}.tmp_traits_embeddings traits, size
    where rand() < (2000 / size.count);

    CREATE OR REPLACE MODEL `{data_set}.kmeans_traits_model`
    OPTIONS(MODEL_TYPE = 'KMEANS', NUM_CLUSTERS = clusters, KMEANS_INIT_METHOD = 'KMEANS++', DISTANCE_TYPE = 'COSINE') AS
    SELECT ml_generate_embedding_result FROM {data_set}.tmp_traits_embeddings_modeling_sample;

    EXECUTE IMMEDIATE ('''CREATE OR REPLACE TABLE `{data_set}.tmp_clustered_traits` AS
        SELECT * FROM ML.PREDICT(MODEL `{data_set}.kmeans_traits_model`,
        (SELECT ml_generate_embedding_result, content, trait_id FROM `{data_set}.tmp_traits_embeddings`))''');

    UPDATE {data_set}.tmp_split_traits_final v set v.cluster_id = cv.centroid_id
    FROM {data_set}.tmp_clustered_traits cv WHERE v.trait_id = cv.trait_id;
END;

Query is running:   0%|          |

""


In [18]:
%%bigquery --pyformat
CREATE OR REPLACE PROCEDURE phase7_name_clusters(trait_type STRING, trait_desc STRING)
BEGIN
    create or replace table {data_set}.tmp_trait_clusters as
    select centroid_id as cluster_id, ARRAY_TO_STRING(ARRAY_AGG(content LIMIT 200), ', ') examples from {data_set}.tmp_clustered_traits group by centroid_id;

    create or replace table {data_set}.tmp_clusters_analysis as
    SELECT AI.GENERATE(FORMAT(p.prompt, trait_desc, c.json_examples), connection_id => 'us.ai_connection', endpoint => 'gemini-2.5-flash-lite').result
    from (select '[\n' || string_agg(json_example, ',\n') || '\n]' json_examples
    from (select cluster_id, TO_JSON_STRING(STRUCT(cluster_id, examples)) json_example from {data_set}.tmp_trait_clusters order by cluster_id)) c
    join {data_set}.prompts p ON p.code = 'cluster_traits';

    create or replace table {data_set}.tmp_cluster_names as
    select CAST(json_value(clusters, '$.cluster_id') AS INT64) cluster_id, json_value(clusters, '$.cluster_name') name, json_value(clusters, '$.cluster_description') description
    from {data_set}.tmp_clusters_analysis, UNNEST(JSON_QUERY_ARRAY(TRIM(result, '`json\n'), '$')) as clusters;

    delete from {data_set}.clusters where cluster_type = trait_type;
    INSERT INTO {data_set}.clusters (cluster_type, cluster_id, name, description)
    SELECT trait_type, cluster_id, name, description FROM {data_set}.tmp_cluster_names;
END;

Query is running:   0%|          |

""


In [19]:
%%bigquery --pyformat
CREATE OR REPLACE PROCEDURE phase7_assign_clusters(trait_type STRING)
BEGIN
    delete from {data_set}.character_cluster_details where book_id IN (SELECT book_id FROM {data_set}.tmp_split_traits_final);
    INSERT INTO {data_set}.character_cluster_details(book_id, id, trait, trait_id, cluster_type, cluster_id)
    SELECT book_id, id, trait, trait_id, trait_type, cluster_id FROM {data_set}.tmp_split_traits_final;

    create or replace table {data_set}.tmp_combined_traits as
    SELECT book_id, id, cluster_type, string_agg(trait, ', ') traits, CAST(NULL AS INT64) cluster_id
    FROM {data_set}.character_cluster_details GROUP BY book_id, id, cluster_type;

    create or replace table {data_set}.tmp_combined_traits_embeddings as SELECT * FROM ML.GENERATE_EMBEDDING(
    MODEL `{data_set}.gemini-embedding-001`,
    (SELECT traits as content, book_id, id FROM {data_set}.tmp_combined_traits),
    STRUCT(TRUE AS flatten_json_output, 'SEMANTIC_SIMILARITY' as task_type, 128 as OUTPUT_DIMENSIONALITY));

    EXECUTE IMMEDIATE ('''CREATE OR REPLACE TABLE `{data_set}.tmp_combined_traits_clustered` AS
        SELECT * FROM ML.PREDICT(MODEL `{data_set}.kmeans_traits_model`,
        (SELECT ml_generate_embedding_result, book_id, id FROM `{data_set}.tmp_combined_traits_embeddings`))''');

    UPDATE {data_set}.tmp_combined_traits ct set ct.cluster_id = ctc.centroid_id
    FROM {data_set}.tmp_combined_traits_clustered ctc WHERE ctc.id = ct.id AND ctc.book_id = ct.book_id;
END;

Query is running:   0%|          |

""
